# Visualize Processing Stages

Notebook to plot multiple mass channels at different stages of low-level processing for each point in each given slide.

Accepted format for images: IonPath's __MIBITiff__ format from MIBI/O or MIBITracker

Please update the __input parameters__ as needed

## load headers

In [114]:
# magic commands
%matplotlib notebook
%load_ext autoreload
%autoreload 2

#imports
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pathlib
import glob
import re

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from mibidata import mibi_image as mi, tiff
import visualize_data as viz

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## input parameters

In [115]:
data_path = pathlib.Path('/Volumes/DISK_IMG/')
save_path = pathlib.Path('/Users/adam_kagel/Desktop/comp/')

mass_channels = [89, 113, 146, 158, 175, 176, 115]
bg_channel = 197

# Anonymized by default currently
# TODO: Read panel if anonymize is false, and use target names
#anonymize_targets = True
#panel_path = data_path.joinpath('Panel54_MelanomaCohort_2.csv')

slide_foldernames = [
    '191204_CohortSlide9_S14-47859-A2-S14-20352-A8',
    '191205_CohortSlide7_S146936A2_G1471023A1_run',
    '191206_CohortSlide11_TMA443_run',
    '191211_CohortSlide10_S158066A1_SP153393_G1675892_run'
]

# leave empty to use full slide name
slide_names = [
    'S9',
    'S7',
    'S11',
    'S10'
]

process_folder_prefix = ''
process_folder_suffix = '_TIFF'

raw_tiff_folder_name = 'bg_none'
proccessed_tiff_folder_name = 'bg_au_050_ta_020'

# process 'prefix' is PointN
process_suffix = '_RowNumber0_Depth_Profile0'
isobar_suffix = '-MassCorrected'
denoise_suffix = '-Filtered'

# Show only given mass channels
#show_raw = mass_channels
#show_bg_cor = mass_channels
#show_iso = [115]
#show_filter = mass_channels

## plot images

In [116]:
plt.ioff()
plt.style.use('dark_background')

def plot_text(ax,text,fontsize):
    ax.text(0.5,0.5,text,verticalalignment='center',horizontalalignment='center',transform=ax.transAxes, color='yellow', fontsize=fontsize)
    ax.set_frame_on(False)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

def make_fig(tiffs,slide,point,channel,titles):
    ncols = len(tiffs)+1
    
    plt.subplots_adjust(hspace=0.0,wspace=0.0)
    wr = [1] + ([3] * (ncols-1))
    gs = gridspec.GridSpec(3,ncols,height_ratios = [1,3,3],width_ratios=wr, hspace=0)
    
    # TODO: too many if/elif/else's
    for n, cell in enumerate(gs):
        ax = plt.subplot(cell)
        if n//ncols == 0:
            plot_text(ax,titles[n],14)
        elif n//ncols == 1:
            if n%ncols ==0:
                plot_text(ax,str(channel),14)
            else:
                image = tiff.read(str(tiffs[n%ncols-1]))
                im = image[channel]
                viz.plot_image(im, ax=ax, brighten_image=True, hi_res=True)
        else:
            if n%ncols == 0:
                plot_text(ax,str(bg_channel),14)
            elif n%ncols == 1:
                image = tiff.read(str(tiffs[0]))
                im = image[bg_channel]
                viz.plot_image(im, ax=ax, brighten_image=True, hi_res=True)
            else:
                ax.axis('off')
        
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        
    if not save_path.is_dir():
        save_path.mkdir()
    comp_dir_name = f'{slide}-{point}'
    comp_dir_path = save_path.joinpath(comp_dir_name)
    if not comp_dir_path.is_dir():
        comp_dir_path.mkdir()
    
    plt.gcf().set_figheight(10)
    plt.gcf().set_figwidth(13.333)
    
    plt.subplots_adjust(hspace=0.0,wspace=0.0)
    plt.savefig(str(comp_dir_path.joinpath(f'{channel}.tiff')), bbox_inches='tight', dpi=300, pad_inches=0.0)
    
    

for slide_n, slide in enumerate(slide_foldernames):
    tiffpath = data_path.joinpath(slide).joinpath(process_folder_prefix+slide+process_folder_suffix)
    rawpath = tiffpath.joinpath(raw_tiff_folder_name)
    #print(slide)
    for rawfilename in glob.glob(f'{str(rawpath)}/*{process_suffix}.tiff'):
        pointNum = int(re.search(r'\d+',pathlib.Path(rawfilename).name).group())
        
        proc_tiffpath = tiffpath.joinpath(proccessed_tiff_folder_name)
        
        tiffs = [
            rawpath.joinpath(rawfilename),
            proc_tiffpath.joinpath(f'Point{pointNum}{process_suffix}.tiff'),
            #proc_tiffpath.joinpath(f'Point{pointNum}{process_suffix}{isobar_suffix}.tiff'),
            proc_tiffpath.joinpath(f'Point{pointNum}{process_suffix}{isobar_suffix}{denoise_suffix}.tiff')
        ]
        
        titles1 = [
            'Mass Channel',
            'Raw Image',
            'Background Removed',
            'Denoised'
        ]
        titles2 = [
            'Mass Channel',
            'Raw Image',
            'Background Removed',
            'Isobaric Correction',
            'Denoised'
        ]
        
        # TODO: programatically select which tiffs to plot for each channel, from input parameters
        for channel in mass_channels:
            if channel != 115:
                make_fig(tiffs,slide_names[slide_n],pointNum,channel,titles1)
            else:
                tiffs2 = tiffs.copy()
                tiffs2.insert(2,proc_tiffpath.joinpath(f'Point{pointNum}{process_suffix}{isobar_suffix}.tiff'))
                make_fig(tiffs2,slide_names[slide_n],pointNum,channel,titles2)
            #break #debug
                
            
        #break #debug
    #break #debug

counts: 2841038.0
counts: 2834685.0
counts: 2648055.0
counts: 1074661.0
counts: 520826.0
counts: 516423.0
counts: 288624.0
counts: 1074661.0
counts: 26552.0
counts: 26168.0
counts: 739.0
counts: 1074661.0
counts: 57739.0
counts: 57075.0
counts: 16748.0
counts: 1074661.0
counts: 237504.0
counts: 233352.0
counts: 132585.0
counts: 1074661.0
counts: 332107.0
counts: 327302.0
counts: 197539.0
counts: 1074661.0
counts: 417951.0
counts: 408765.0
counts: 405650.0
counts: 215459.0
counts: 1074661.0


/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "fov_id" attribute is required if "folder" is specified. Setting "fov_id" to Point4.
  f'The "{field_name}" attribute is required if "{value_name}" '
/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "folder" attribute is required if "fov_id" is specified. Setting "folder" to Point4.
  f'The "{field_name}" attribute is required if "{value_name}" '


counts: 49537.0
counts: 49537.0


/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "fov_id" attribute is required if "folder" is specified. Setting "fov_id" to Point4-MassCorrected-Filtered.
  f'The "{field_name}" attribute is required if "{value_name}" '
/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "folder" attribute is required if "fov_id" is specified. Setting "folder" to Point4-MassCorrected-Filtered.
  f'The "{field_name}" attribute is required if "{value_name}" '


counts: 42471.0
counts: 560.0
counts: 7588.0
counts: 7588.0
counts: 2299.0
counts: 560.0
counts: 396.0
counts: 396.0
counts: 153.0
counts: 560.0
counts: 1425.0
counts: 1425.0
counts: 786.0
counts: 560.0
counts: 4329.0
counts: 4329.0
counts: 1678.0
counts: 560.0
counts: 6214.0
counts: 6214.0
counts: 2526.0
counts: 560.0
counts: 3231.0
counts: 3231.0


...

counts: 1515949.0
counts: 1448546.0


/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "fov_id" attribute is required if "folder" is specified. Setting "fov_id" to Point3-MassCorrected-Filtered.
  f'The "{field_name}" attribute is required if "{value_name}" '
/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "folder" attribute is required if "fov_id" is specified. Setting "folder" to Point3-MassCorrected-Filtered.
  f'The "{field_name}" attribute is required if "{value_name}" '


counts: 1340013.0
counts: 22210016.0
counts: 580260.0
counts: 510302.0
counts: 412406.0
counts: 22210016.0
counts: 29118.0
counts: 17501.0
counts: 7378.0
counts: 22210016.0
counts: 56697.0
counts: 44722.0
counts: 28769.0
counts: 22210016.0
counts: 854331.0
counts: 771915.0
counts: 698766.0
counts: 22210016.0
counts: 181034.0
counts: 154044.0
counts: 112361.0
counts: 22210016.0
counts: 420378.0
counts: 252987.0


/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "fov_id" attribute is required if "folder" is specified. Setting "fov_id" to Point3-MassCorrected.
  f'The "{field_name}" attribute is required if "{value_name}" '
/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "folder" attribute is required if "fov_id" is specified. Setting "folder" to Point3-MassCorrected.
  f'The "{field_name}" attribute is required if "{value_name}" '


counts: 190737.0
counts: 120808.0
counts: 22210016.0
counts: 6125746.0
counts: 6113202.0
counts: 5888075.0
counts: 2286850.0
counts: 1082579.0
counts: 1072046.0
counts: 898626.0
counts: 2286850.0
counts: 47010.0
counts: 44751.0
counts: 978.0
counts: 2286850.0
counts: 61329.0
counts: 59478.0
counts: 4862.0
counts: 2286850.0
counts: 1903653.0
counts: 1893808.0
counts: 1725562.0
counts: 2286850.0
counts: 1059727.0
counts: 1040935.0
counts: 870472.0
counts: 2286850.0
counts: 626108.0
counts: 598944.0
counts: 592506.0
counts: 387621.0
counts: 2286850.0
